<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/top.png?raw=true'>

# Configuration (à lancer avant tous les notebooks)

In [ ]:
# version de python
import platform
platform.python_version()

In [ ]:
# la liste des packages installés
!conda list

In [ ]:
# version de la SDK azureml
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Si le notebook est executé en dehors d'Azure, il faut télécharger le fichier config.json depuis le portail https://portal.azure.com/, et le mettre dans le workspace qui contient le notebook.

Si le notebook est exécuté directement depuis le workspace Azure, le fichier de config devrait déjà être là.

In [ ]:
# connexion au workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

# Lancer une experience

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment.experiment?view=azure-ml-py

    An Experiment is a container of trials that represent multiple model runs.

In [ ]:
from azureml.core import Experiment

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "titanic-explore-experiment")

# Start logging data from the experiment
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

## logger des infos

In [ ]:
import pandas as pd

# load the data from a local file
titanic = pd.read_csv('data/titanic/train.csv')

titanic.head()

In [ ]:
# Count the rows and log the result
row_count = (len(titanic))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

In [ ]:
#!pip install seaborn --upgrade

In [ ]:
# log le nombre de survivants par classe
import seaborn as sns

fig = sns.catplot(x = "Survived", kind = "count", col = "Pclass", data =titanic)

run.log_image(name = 'distribution par classe', plot = fig)

## sauver un fichier csv

In [ ]:
titanic.sample(100)

In [ ]:
# Car les noms contiennent des "," et des " ", pas pratique pour le csv
titanic.drop("Name", inplace=True, axis=1)

In [ ]:
# Save a sample of the data and upload it to the experiment output
titanic.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name = 'outputs/sample.csv', path_or_stream = 'sample.csv')

In [ ]:
# Complete the run
run.complete()

## résultats

In [ ]:
import json

# Get run details
details = run.get_details()
print(details)

# Get logged metrics
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

# Get output files
files = run.get_file_names()
print(json.dumps(files, indent=2))

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

# Lancer une expérience avec un script

## création d'un dossier qui contient le script et les données

In [ ]:
import os, shutil

# Create a folder for the experiment files
folder_name = 'titanic-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/titanic/train.csv', os.path.join(folder_name, "titanic.csv"))

le code suivant va créer un script titanic_experiment.py :

In [ ]:
%%writefile $folder_name/titanic-experiment.py

from azureml.core import Run
import pandas as pd

# Get the experiment run context
run = Run.get_context()

# load the data from a local file
titanic = pd.read_csv('titanic.csv')

# Count the rows and log the result
row_count = (len(titanic))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Save a sample of the data and upload it to the experiment output
titanic.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name = 'outputs/sample.csv', path_or_stream = 'sample.csv')

# Complete the run
run.complete()

## execution de l'expérience

In [ ]:
import os
import sys
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig
from azureml.widgets import RunDetails

# create a new RunConfig object
experiment_run_config = RunConfiguration()

# Create a script config
src = ScriptRunConfig(source_directory=experiment_folder, 
                      script='titanic-experiment.py',
                      run_config=experiment_run_config) 

# submit the experiment
experiment = Experiment(workspace = ws, name = 'titanic-experiment')
run = experiment.submit(config=src)
RunDetails(run).show()
run.wait_for_completion()

# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/down.png?raw=true'>



In [ ]:
# stop toutes les instances de calcul
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state == 'Running':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

In [ ]:
# liste tous les compute pour vérifier qu'elles sont éteintes
for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance:
        print(compute.name, compute.get_status())

# Ressources

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)